In [13]:
import requests

from ape import networks, Contract
import pandas as pd

from helpers.coingecko import get_prices
from helpers.subgraph_endpoints import *
from helpers.aura import get_gravi_in_balancer_pool, weekly_emissions_after_fee, aura_mint_ratio
from helpers.vp_info import get_council_vp_fee, get_voter_vp
from helpers.bribe_info import get_bribe_info, MAX_BRIBE, GAUGE_CAP
from helpers.formatter import pct_format, dollar_format

In [ ]:
networks.parse_network_choice("ethereum:mainnet").__enter__()

In [15]:
bal_price, aura_price,  badger_price, rpl_price = get_prices()
data = [["${:.2f}".format(badger_price),  "${:.2f}".format(bal_price), "${:.2f}".format(aura_price), "${:.2f}".format(rpl_price)]]
headers = ["Badger price", "Balancer price", "Aura price", "Rpl price"]
df_prices = pd.DataFrame(data, columns=headers)
df_prices

,Badger price,Balancer price,Aura price,Rpl price
0,$2.45,$5.24,$2.02,$15.60


In [16]:
# NOTE: modify with right proposal (latest & current)
proposal_id_last = "0xe22068244d488910cf46d4ae6b4c94cb61bea81981f5467936cd72757f90f2bd"
proposal_id_current = "0x965fb50ce65ca8ee973e8d00e896c70e5a78de075b8703bc43079bbf27191301"

cost_per_vote_after_fee = get_cost_per_vote_after_fee()

# check active voter in past proposal as ref
response_last_proposal = get_proposal_info(proposal_id_last)
response_current_proposal = get_proposal_info(proposal_id_current)

block_current_proposal = int(response_current_proposal["snapshot"])
aura_voting_actively_last_round = response_last_proposal["scores_total"]
data = [[block_current_proposal,  f'{aura_voting_actively_last_round:.0f}', dollar_format(cost_per_vote_after_fee)]]
headers = ["Proposal block height", "Past Round Active vlAURA votes", "Cost per Vote Last Round"]
df_llama = pd.DataFrame(data, columns=headers)
df_llama


,Proposal block height,Past Round Active vlAURA votes,Cost per Vote Last Round
0,15936810,7602141,$0.0633


In [17]:
# emissions ecosystem
aura = Contract('0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF')
aura_mint_ratio = aura_mint_ratio(aura, block_current_proposal)
weekly_emissions_after_fee = weekly_emissions_after_fee(aura_mint_ratio, bal_price, aura_price)
biweekly_emissions = weekly_emissions_after_fee * 2
data = [[aura_mint_ratio, dollar_format(biweekly_emissions)]]
headers = ["Aura Mint Balancer Ratio", "Bi-weekly Emissions"]
df_emissions = pd.DataFrame(data, columns=headers)
df_emissions

,Aura Mint Balancer Ratio,Bi-weekly Emissions
0,3.677744,$2755516.9673


In [18]:
vlAURA = Contract("0x3Fa73f1E5d8A792C80F426fc8F84FBF7Ce9bBCAC")
vlAURA_ts = vlAURA.totalSupply(block_identifier=block_current_proposal)/1e18
council_fee = get_council_vp_fee(vlAURA, block_current_proposal)

# digg pool
balancer_vault = Contract("0xBA12222222228d8Ba445958a75a0704d566BF2C8")
gravi_in_digg_pool = get_gravi_in_balancer_pool(balancer_vault, block_current_proposal)
total_gravi_vp_for_digg = gravi_in_digg_pool + council_fee

treasury_votes = get_voter_vp(vlAURA, block_current_proposal)
total_vp_badgerdao = treasury_votes + council_fee

pct_controlled_aura_by_badger_treasury = treasury_votes / vlAURA_ts
pct_controlled_aura_by_badger_treasury_active = treasury_votes / aura_voting_actively_last_round
pct_controlled_aura_by_badger = total_vp_badgerdao/ vlAURA_ts
pct_controlled_aura_by_badger_active = total_vp_badgerdao/ aura_voting_actively_last_round
pct_controlled_aura_by_council_active = total_gravi_vp_for_digg / aura_voting_actively_last_round
data = [[treasury_votes, council_fee, total_vp_badgerdao, gravi_in_digg_pool, total_gravi_vp_for_digg, pct_format(pct_controlled_aura_by_badger), pct_format(pct_controlled_aura_by_badger_active)]]
headers = ["Treasury VP", "Council Fee", "Total Badger VP", "Gravi in Digg pool", "Total Badger Council VP - if voting for DIGG", "Badger Aura Controlled - everyone Votes", "Badger Aura Controlled - proactive voters"]
df_vp = pd.DataFrame(data, columns=headers)
df_vp

,Treasury VP,Council Fee,Total Badger VP,Gravi in Digg pool,Total Badger Council VP - if voting for DIGG,Badger Aura Controlled - everyone Votes,Badger Aura Controlled - proactive voters
0,271835.162534,67286.9263,339122.088834,176125.421812,243412.348111,3.480%,4.461%


In [19]:
vebal = Contract("0xC128a9954e6c874eA3d62ce62B468bA073093F25")
vebal_ts = vebal.totalSupplyAt(block_current_proposal) / 1e18
vebal_aura_controlled = vebal.balanceOfAt("0xaF52695E1bB01A16D33D7194C28C42b10e0Dbec2", block_current_proposal)/1e18
pct_aura_vebal_controlled = vebal_aura_controlled/vebal_ts
pct_badger_vebal_controlled = pct_controlled_aura_by_badger*pct_aura_vebal_controlled
pct_badger_vebal_controlled_active = pct_controlled_aura_by_badger_active*pct_aura_vebal_controlled
pct_council_vebal_controlled_active = pct_controlled_aura_by_council_active * pct_aura_vebal_controlled
data = [[pct_format(pct_aura_vebal_controlled), pct_format(pct_badger_vebal_controlled), pct_format(pct_badger_vebal_controlled_active)]]
headers = ["Aura veBAL controlled", "Badger veBAL controlled - everyone votes", "Badger veBAL controlled - proactive voters"]
df_vebal = pd.DataFrame(data, columns=headers)
df_vebal

,Aura veBAL controlled,Badger veBAL controlled - everyone votes,Badger veBAL controlled - proactive voters
0,24.857%,0.865%,1.109%


In [20]:
# NOTE: block vlaura/vebal relative weight breakdown for badger/wbtc cap
vlAURA_voting_badger_wbtc = treasury_votes
vebal_expected_relative_weight_badger_wbtc = pct_controlled_aura_by_badger_treasury_active * pct_aura_vebal_controlled
data = [[vlAURA_voting_badger_wbtc, pct_format(vebal_expected_relative_weight_badger_wbtc)]]
headers = ["vlAURA badger/wbtc pool", "veBAL relative weight"]
df_bribes_badger_pool = pd.DataFrame(data, columns=headers)
df_bribes_badger_pool

,vlAURA badger/wbtc pool,veBAL relative weight
0,271835.162534,0.889%


In [21]:
# NOTE: block vlaura/vebal relative weight breakdown for digg/wbtc/gravi cap
vlAURA_voting_digg_pool = total_gravi_vp_for_digg
vebal_expected_relative_weight_digg = pct_controlled_aura_by_council_active * pct_aura_vebal_controlled
data = [[vlAURA_voting_digg_pool, pct_format(vebal_expected_relative_weight_digg)]]
headers = ["vlAURA DIGG pool", "veBAL relative weight"]
df_bribes_digg_pool = pd.DataFrame(data, columns=headers)
df_bribes_digg_pool

,vlAURA DIGG pool,veBAL relative weight
0,243412.348111,0.796%


In [22]:
# bribes allocations
bribe_for_reth_badger = 10_000
bribe_for_badgerfraxbp = 6_000
bribe_for_reth_badger_from_rocket = 225
data = [[bribe_for_reth_badger, bribe_for_badgerfraxbp, bribe_for_reth_badger_from_rocket]]
headers = ["Bribe allocated to badger/reth pool (BADGER)", "Bribe allocated to badgerfraxbp pool", "Bribe allocated to badger/reth pool (ROCKET)"]
df_bribes_allocation = pd.DataFrame(data, columns=headers)
df_bribes_allocation


,Bribe allocated to badger/reth pool (BADGER),Bribe allocated to badgerfraxbp pool,Bribe allocated to badger/reth pool (ROCKET)
0,10000,6000,225


In [23]:
# badger/reth prediction for vebal relative weight given the bribe
bribe_badgereth_usd = bribe_for_reth_badger * badger_price + bribe_for_reth_badger_from_rocket * rpl_price
vlAURA_votes_bought = bribe_badgereth_usd / cost_per_vote_after_fee
pct_controlled_aura_by_bribing_badger_reth = vlAURA_votes_bought / aura_voting_actively_last_round
vebal_expected_relative_weight_badger_reth = pct_controlled_aura_by_bribing_badger_reth * pct_aura_vebal_controlled
data = [[vlAURA_votes_bought, pct_format(vebal_expected_relative_weight_badger_reth), dollar_format(bribe_badgereth_usd)]]
headers = ["vlAURA badger/reth pool", "veBAL relative weight", "Bribe Cost"]
df_bribe_breakdown_badgereth = pd.DataFrame(data, columns=headers)
df_bribe_breakdown_badgereth

,vlAURA badger/reth pool,veBAL relative weight,Bribe Cost
0,442399.4533,1.447%,$28010.0000


In [24]:
# treasury expected earnings bi-weekly expectation post-bribe in BADGER/WBTC pool
tvl_badger = get_tvl_balancer_badger_pool()
badger_pool = Contract("0xb460DAa847c45f1C4a41cb05BFB3b51c92e41B36")
pool_supply = badger_pool.totalSupply() / 1e18
reward_badger_aura = Contract("0x05df1E87f41F793D9e03d341Cdc315b76595654C")
vault_rewards_contract_balance = reward_badger_aura.balanceOf("0xD0A7A8B98957b9CD3cFB9c0425AbE44551158e9e", block_identifier=block_current_proposal)/1e18
vault_owned_tcl = tvl_badger * (vault_rewards_contract_balance/pool_supply)
pool_capture = vault_owned_tcl / tvl_badger
pool_earnings = biweekly_emissions * vebal_expected_relative_weight_badger_wbtc
data = [[dollar_format(pool_earnings), dollar_format(tvl_badger), pct_format(pool_capture), dollar_format(pool_earnings*pool_capture)]]
headers = ["Pool yield (USD) - biweekly", "BADGER Pool TVL", "BADGER Pool Capture", "Treasury yield earned (USD) - biweekly"]
df_earnings_badger_pool = pd.DataFrame(data, columns=headers)
df_earnings_badger_pool

,Pool yield (USD) - biweekly,BADGER Pool TVL,BADGER Pool Capture,Treasury yield earned (USD) - biweekly
0,$24491.9123,$5109867.8329,39.967%,$9788.7963


In [25]:
# treasury expected earnings bi-weekly expectation in DIGG pool
tvl_digg = get_tvl_balancer_digg_pool()
digg_pool = Contract("0x8eB6c82C3081bBBd45DcAC5afA631aaC53478b7C")
pool_supply = digg_pool.totalSupply() / 1e18
reward_badger_aura = Contract("0xe86f0312b06126855810b4a13a43c3e2b1b8dd90")
vault_rewards_contract_balance = reward_badger_aura.balanceOf("0xD0A7A8B98957b9CD3cFB9c0425AbE44551158e9e", block_identifier=block_current_proposal)/1e18
vault_owned_tcl = tvl_digg * (vault_rewards_contract_balance/pool_supply)
pool_capture = vault_owned_tcl / tvl_digg
pool_earnings = biweekly_emissions * vebal_expected_relative_weight_digg
data = [[dollar_format(pool_earnings), dollar_format(tvl_digg), pct_format(pool_capture), dollar_format(pool_earnings*pool_capture)]]
headers = ["Pool yield (USD) - biweekly", "DIGG Pool TVL", "DIGG Pool Capture", "Treasury yield earned (USD) - biweekly"]
df_earnings_badger_pool = pd.DataFrame(data, columns=headers)
df_earnings_badger_pool

,Pool yield (USD) - biweekly,DIGG Pool TVL,DIGG Pool Capture,Treasury yield earned (USD) - biweekly
0,$21931.0623,$1845224.3499,40.057%,$8784.8546


In [26]:
# treasury expected earnings bi-weekly expectation post-bribe into BADGER/RETH pool
tvl_rethbadger = get_tvl_balancer_badgerreth_pool()
badgereth_pool = Contract("0x1E19CF2D73a72Ef1332C882F20534B6519Be0276")
pool_supply = badgereth_pool.totalSupply() / 1e18
reward_badger_aura = Contract("0x685c94e7da6c8f14ae58f168c942fb05bad73412")
vault_rewards_contract_balance = reward_badger_aura.balanceOf("0xD0A7A8B98957b9CD3cFB9c0425AbE44551158e9e", block_identifier=block_current_proposal)/1e18
vault_owned_tcl = tvl_digg * (vault_rewards_contract_balance/pool_supply)
pool_capture = vault_owned_tcl / tvl_digg
pool_earnings = biweekly_emissions * vebal_expected_relative_weight_badger_reth
treasury_roi = (pool_earnings*pool_capture) / bribe_badgereth_usd
data = [[dollar_format(pool_earnings), dollar_format(tvl_rethbadger), pct_format(pool_capture), dollar_format(pool_earnings*pool_capture), pct_format(treasury_roi)]]
headers = ["Pool yield (USD) - biweekly", "BADGER/RETH Pool TVL", "BADGER/RETH Pool Capture", "Treasury yield earned (USD) - biweekly", "Treasury ROI - biweekly"]
df_earnings_badger_pool = pd.DataFrame(data, columns=headers)
df_earnings_badger_pool


,Pool yield (USD) - biweekly,BADGER/RETH Pool TVL,BADGER/RETH Pool Capture,Treasury yield earned (USD) - biweekly,Treasury ROI - biweekly
0,$39859.4815,$2279251.5304,74.274%,$29605.2329,105.695%


In [27]:
# NOTE: block to max out badger/wbtc cap
vlAURA_req, cost_of_max_cap, badger_to_bribe_badger_pool = get_bribe_info(treasury_votes, pct_controlled_aura_by_badger_treasury_active*pct_aura_vebal_controlled, cost_per_vote_after_fee,badger_price)
bribe_reclycing_badger_voting = badger_to_bribe_badger_pool * (treasury_votes/vlAURA_req)
data = [[vlAURA_req, dollar_format(cost_of_max_cap), badger_to_bribe_badger_pool, bribe_reclycing_badger_voting]]
headers = ["vlAURA Required", "Cost of Maxing Cap", "Badger Bribe Max Cap", "Expected Bribe reclyced"]
df_bribes_badger_pool = pd.DataFrame(data, columns=headers)
df_bribes_badger_pool

,vlAURA Required,Cost of Maxing Cap,Badger Bribe Max Cap,Expected Bribe reclyced
0,611668.368348,$38727.0619,15806.964027,7024.866509


In [28]:
# treasury expected earnings bi-weekly expectation post-bribe into BADGER pool
tvl_badger = get_tvl_balancer_badger_pool()
badger_pool = Contract("0xb460DAa847c45f1C4a41cb05BFB3b51c92e41B36")
pool_supply = badger_pool.totalSupply() / 1e18
reward_badger_aura = Contract("0x05df1E87f41F793D9e03d341Cdc315b76595654C")
vault_rewards_contract_balance = reward_badger_aura.balanceOf("0xD0A7A8B98957b9CD3cFB9c0425AbE44551158e9e", block_identifier=block_current_proposal)/1e18
vault_owned_tcl = tvl_badger * (vault_rewards_contract_balance/pool_supply)
pool_capture = vault_owned_tcl / tvl_badger
pool_earnings = biweekly_emissions * GAUGE_CAP
real_bribe_aura_expense = (badger_to_bribe_badger_pool - bribe_reclycing_badger_voting) * badger_price
treasury_roi = (pool_earnings*pool_capture) / real_bribe_aura_expense
data = [[dollar_format(pool_earnings), dollar_format(tvl_badger), pct_format(pool_capture), dollar_format(pool_earnings*pool_capture), dollar_format(real_bribe_aura_expense), pct_format(treasury_roi)]]
headers = ["Pool yield (USD) - biweekly", "BADGER Pool TVL", "BADGER Pool Capture", "Treasury yield earned (USD) - biweekly", "Bribe Aura Real Cost", "Treasury ROI - biweekly"]
df_earnings_badger_pool = pd.DataFrame(data, columns=headers)
df_earnings_badger_pool

,Pool yield (USD) - biweekly,BADGER Pool TVL,BADGER Pool Capture,Treasury yield earned (USD) - biweekly,Bribe Aura Real Cost,Treasury ROI - biweekly
0,$55110.3393,$5109867.8329,39.967%,$22026.2052,$21516.1389,102.371%
